In [20]:
import pandas as pd 
df=pd.read_excel("delivery system/truckdata.xlsx")
W = df["Weekly_Weight_Requirement"]
df=df.head(100)

In [21]:
import gurobipy as gp
from gurobipy import GRB
import csv

num_locations = 100
days = ['M', 'T', 'W', 'R', 'F']
num_days = len(days)

truck_types = [15, 10]   
truck_costs = [150, 120] 
W = df["Weekly_Weight_Requirement"] 

model = gp.Model("TruckDeliveryOptimization") 
x = model.addVars(num_locations, num_days, len(truck_types), vtype=GRB.INTEGER, name="x") 
y = model.addVars(num_locations, 4, vtype=GRB.BINARY, name="y") 

model.setObjectiveN(gp.quicksum(truck_costs[t] * x[i, d, t] 
                               for i in range(num_locations) 
                               for d in range(num_days) 
                               for t in range(len(truck_types))), priority=1, index=0, weight=1) 

for i in range(num_locations):
    weight = (W[i] // 150) * 150
    model.addConstr(gp.quicksum(truck_types[t] * x[i, d, t] 
                                for d in range(num_days) 
                                for t in range(len(truck_types))) >= W[i], 
                    name=f"weekly_weight_{i}")

for i in range(num_locations):
    model.addConstr(gp.quicksum(y[i, s] for s in range(4)) == 1, name=f"schedule_selection_{i}")

patterns = {
    0: ['M', 'T', 'W', 'R', 'F'],  
    1: ['M'],                      
    2: ['M', 'R'],                 
    3: ['M', 'W', 'F']            
}

day_indices = {day: i for i, day in enumerate(days)}

for i in range(num_locations):
    total_weight = gp.quicksum(truck_types[t] * x[i, d, t] 
                                for d in range(num_days) 
                                for t in range(len(truck_types)))

    # Once a week
    for d in patterns[1]:
        model.addGenConstrIndicator(y[i, 1], True,
            gp.quicksum(truck_types[t] * x[i, day_indices[d], t] for t in range(len(truck_types))) == total_weight, name=f"once_a_week_{i}")

    # Twice a week
    for d in patterns[2]:
        model.addGenConstrIndicator(y[i, 2], True,
            gp.quicksum(truck_types[t] * x[i, day_indices[d], t] for t in range(len(truck_types))) == total_weight / 2, name=f"twice_a_week_{i}")

    # Thrice a week
    for d in patterns[3]:
        model.addGenConstrIndicator(y[i, 3], True,
            gp.quicksum(truck_types[t] * x[i, day_indices[d], t] for t in range(len(truck_types))) == total_weight / 3, name=f"thrice_a_week_{i}")

    # Five times a week
    for d in patterns[0]:
        model.addGenConstrIndicator(y[i, 0], True,
            gp.quicksum(truck_types[t] * x[i, day_indices[d], t] for t in range(len(truck_types))) == total_weight / 5, name=f"five_times_a_week_{i}")

min_10 = model.addVar(vtype=GRB.INTEGER, lb=0, name="min_trucks_10")
min_15 = model.addVar(vtype=GRB.INTEGER, lb=0, name="min_trucks_15")

for j in range(num_days):
    model.addConstr(min_10 >= gp.quicksum(x[i, j, 1] for i in range(num_locations)))
    model.addConstr(min_15 >= gp.quicksum(x[i, j, 0] for i in range(num_locations))-gp.quicksum(x[i, j, 1] for i in range(num_locations)))
    model.addConstr(min_15 >= -gp.quicksum(x[i, j, 0] for i in range(num_locations))+gp.quicksum(x[i, j, 1] for i in range(num_locations)))

model.setObjectiveN(min_15 , priority=2, index=0, weight=1)

model.optimize()

if model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Finding IIS...")
    model.computeIIS()
    model.write("infeasible.ilp")
elif model.status == GRB.OPTIMAL:
    with open('optimization_results.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Location", "Day", "Truck Type (tons)", "Number of Trucks", "Delivery Schedule"])

        for i in range(num_locations):
            # for d in range(num_days):
            #     for t in range(len(truck_types)):
            #         if x[i, d, t].x > 0:
            #             writer.writerow([i+1, days[d], truck_types[t], int(x[i, d, t].x), ""])

            for s in range(4):
                if y[i, s].x > 0:
                    schedule_type = ["Five times a week", "Once a week", "Twice a week", "Thrice a week"]
                    writer.writerow([i+1, "", "", "", schedule_type[s]])

    print("Results saved to 'optimization_results.csv'.")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 215 rows, 1402 columns and 3915 nonzeros
Model fingerprint: 0x3eead1d4
Model has 1100 general constraints
Variable types: 0 continuous, 1402 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
  GenCon coe range [2e+00, 2e+01]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 1 objectives ... 
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 () ...
----------------------------------------